In [1]:
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import TextLoader

# 문서를 로드
documents = TextLoader("../Assets/data/사우회관리규정.txt").load()

[RecursiveCharacterTextSplitter](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/
) 를 사용하여 문서를 청크로 분할 

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap,
    )
    docs = text_splitter.split_documents(documents)
    return docs
  
# docs 변수에 분할 문서를 저장
docs = split_docs(documents)

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings를 사용하여 문장을 벡터로 변환
# 문장을 벡터로 변환하고 이를 통해 텍스트 데이터의 의미적 유사성을 계산하기 위해 사용
embeddings = OpenAIEmbeddings()

# Chromdb에 벡터를 저장하고 유사도 검색을 지원
vectorstore = Chroma.from_documents(docs, embeddings)

# 검색기를 사용하여 문서를 검색하고 형식을 지정
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

In [5]:
from langchain_openai import ChatOpenAI

# llm API를 호출
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # gpt-3.5-turbo 모델 사용
    temperature=0,  # 창의성 0으로 설정
)

In [6]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def build_chain(retriever, llm):
    # 프롬프트 모듈을 가져와서 사용
    prompt = hub.pull("rlm/rag-prompt")

    # 체인을 구성
    chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain
  
# 체인을 구성
chain = build_chain(retriever, llm)

# 체인을 호출
completion = chain.invoke("사우회관리규정에 대해 설명해주세요.")
print(completion)

사우회는 주식회사 사우회로 불리며 회원의 상호부조와 복지향상을 목적으로 합니다. 사우회는 상조사업 및 이 회의 목적을 달성하기 위한 기타 사업을 행합니다. 회비는 매월 급여 지급시 월 10,000원을 공제하며, 근속년수에 따라 지급 비율이 달라집니다.


In [7]:
# 벡터 저장소 삭제
vectorstore.delete_collection()